In [ ]:
# TF_data.py
# Proyecto Bike Buyers - CRISP-DM completo en un solo archivo

import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tkinter import Tk, filedialog

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# 1. CARGA INTERACTIVA DEL CSV
REPO_ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
DEFAULT_CSV_PATH = REPO_ROOT / "data.csv"

def cargar_csv_interactivo():
    """
    Intenta cargar automáticamente el archivo bike_buyers.csv desde la ruta del repositorio.
    Si no existe, abre una ventana para seleccionarlo manualmente.
    """
    if DEFAULT_CSV_PATH.exists():
        print(f"\n✅ Archivo cargado automáticamente: {DEFAULT_CSV_PATH}\n")
        return pd.read_csv(DEFAULT_CSV_PATH)

    print("Se abrirá una ventana para que elijas el archivo bike_buyers.csv...")
    root = Tk()
    root.withdraw()  # Oculta la ventana principal
    root.attributes("-topmost", True)  # La pone delante de todo

    ruta = filedialog.askopenfilename(
        title="Selecciona el archivo bike_buyers.csv",
        filetypes=[("CSV files", "*.csv"), ("Todos los archivos", "*.*")]
    )

    root.destroy()

    if not ruta:
        raise SystemExit("❌ No seleccionaste ningún archivo. Vuelve a ejecutar el programa.")

    print(f"\n✅ Archivo seleccionado: {ruta}\n")
    return pd.read_csv(ruta)

df = cargar_csv_interactivo()

KeyboardInterrupt: 

In [ ]:
# 2. CARGAR Y LIMPIAR DATA
print("Primeras filas del dataset:")
print(df.head())
print("\nTamaño del dataset:", df.shape)
print("\nNulos por columna:")
print(df.isna().sum())

df_clean = df.dropna().copy()
print("\nTamaño después de dropna():", df_clean.shape)

In [ ]:
# 3. NUEVAS VARIABLES
df_clean["Con_hijos"] = np.where(df_clean["Children"] > 0, "Si", "No")
df_clean["Con_vehiculo"] = np.where(df_clean["Cars"] > 0, "Si", "No")

print("\nFrecuencia Con_hijos:")
print(df_clean["Con_hijos"].value_counts())
print("\nFrecuencia Con_vehiculo:")
print(df_clean["Con_vehiculo"].value_counts())

In [ ]:
# 4. REQUERIMIENTOS (PREGUNTAS)
print("\n" + "=" * 60)
print("PREGUNTA 1: Promedio de ingresos según si compró bicicleta")
print("=" * 60)
q1 = df_clean.groupby("Purchased Bike")["Income"].mean()
print(q1)

print("\n" + "=" * 60)
print("PREGUNTA 2: Promedio de ingresos según estado civil")
print("=" * 60)
q2 = df_clean.groupby("Marital Status")["Income"].mean()
print(q2)

print("\n" + "=" * 60)
print("PREGUNTA 3: Promedio de hijos según nivel educativo (solo con hijos)")
print("=" * 60)
q3 = df_clean[df_clean["Con_hijos"] == "Si"].groupby("Education")["Children"].mean()
print(q3)

print("\n" + "=" * 60)
print("PREGUNTA 4: Promedio de vehículos según ocupación (solo con vehículo)")
print("=" * 60)
q4 = df_clean[df_clean["Con_vehiculo"] == "Si"].groupby("Occupation")["Cars"].mean()
print(q4)

print("\n" + "=" * 60)
print("PREGUNTA 5: Promedio de edad según si es propietario de vivienda")
print("=" * 60)
q5 = df_clean.groupby("Home Owner")["Age"].mean()
print(q5)

plt.figure()
q1.plot(kind="bar")
plt.title("Ingreso promedio según compra de bicicleta")
plt.xlabel("Purchased Bike")
plt.ylabel("Ingreso promedio")
plt.tight_layout()
plt.show()

In [ ]:
# 5. MODELADO (SCORING)
print("\n" + "=" * 60)
print("MODELADO: Regresión Logística para predecir 'Purchased Bike'")
print("=" * 60)

df_model = df_clean.copy()
df_model["Target"] = np.where(df_model["Purchased Bike"] == "Yes", 1, 0)

y = df_model["Target"]
X = df_model.drop(columns=["Purchased Bike", "Target", "ID"])

numeric = ["Income", "Children", "Cars", "Age"]
categorical = [c for c in X.columns if c not in numeric]

print("\nVariables numéricas:", numeric)
print("Variables categóricas:", categorical)

preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical)
    ]
)

model = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=1000))
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print("\nResultados del modelo:")
print(f"Accuracy: {acc:.4f}")
print(f"AUC-ROC: {auc:.4f}")
print("\nClassification report:")
print(classification_report(y_test, y_pred))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
plt.figure()
plt.plot(fpr, tpr, label=f"ROC (AUC = {auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC - Regresión Logística")
plt.legend()
plt.tight_layout()
plt.show()

print("\nPROFEE PONGANOS 20 :D .")